<a href="https://colab.research.google.com/github/Syunkolee9891/mario-test/blob/master/Mario_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 前準備

まずはGoogle Colabの環境を確認します.

## 環境の確認

上のメニューの「ランタイム」から

* 「ランタイムのタイプを変更」

を選んでハードウェアアクセラレータに

* 「GPU」

が選ばれていることを確認してください.

In [ ]:
# 環境の確認
!echo -- OS --
!cat /etc/os-release | grep -e ^VERSION= -e ^NAME=
!echo
!echo -- python --
!echo python version: `python --version`
!echo
!echo -- GPU --
!nvidia-smi

## Google Driveのマウント

次に, Google Driveをマウントします.

12時間以上使うとランタイム上のデータごと消えてしまうのでGoogle Drive上にデータを保存し, 学習データ等を保存していきます.

In [ ]:
# Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive')

## ユーティリティの準備

今後コマンド等を実行する際に必要な細々とした変数や関数を定義します.

In [ ]:
# ユーティリティ関数等の定義
import os
DFL_BASE_DIR = "/content/drive/My Drive/DFL"
DFLL_DIR = DFL_BASE_DIR + "/DeepFaceLab_Linux"
DFL_DIR = DFLL_DIR + "/DeepFaceLab"
SCRIPTS_DIR = DFLL_DIR + "/scripts"

!if [ ! -d "{DFL_BASE_DIR}" ]; then mkdir "{DFL_BASE_DIR}"; fi

def run_script(script_name):
    os.chdir(SCRIPTS_DIR)
    !bash {script_name}

上で定義した変数はpython上での変数ですが, `!pwd` のようなシェルコマンドの実行時には `{` と `}` で括ると使うことが出来ます.

In [ ]:
print(DFL_BASE_DIR)
!echo "{DFL_BASE_DIR}"

## ソースコードのダウンロード（クローン）

次に, [**iperov/DeepFaceLab**](https://github.com/iperov/DeepFaceLab) のソースコードをgitでGoogle Drive上に保存します.

**DeepFaceLab** はWindows環境を想定して作られているので今回は [**nagadit/DeepFaceLab_Linux**](https://github.com/nagadit/DeepFaceLab_Linux) という [nagadit](https://github.com/nagadit) 氏が移植してくれたシェルスクリプト群をフォークしてGoogle Colab用にほんの少しだけカスタムしたもの ([2bbb/DeepFaceLab_Linux](https://github.com/2bbb/DeepFaceLab_Linux)) を使います.



In [ ]:
# git clone (Google Driveに保存するので1度だけでOK)
!if [ ! -d "{DFLL_DIR}" ]; then git clone -b colab --recursive https://github.com/2bbb/DeepFaceLab_Linux.git "{DFLL_DIR}" ; else echo "Already cloned" ; fi
!if [ ! -d "{DFLL_DIR}/workspace" ]; then mkdir "{DFLL_DIR}/workspace" ; fi

In [ ]:
# git cloneの結果の確認
!echo -- DeepFaceLab_Linux --
!ls "{DFLL_DIR}"
!echo
!echo -- DeepFaceLab --
!ls "{DFL_DIR}"

コードがDFL上に保存出来たら DeepFaceLab に必要なpythonモジュールをインストールします.

Google ColabのデフォルトのtensorflowのバージョンがDFLの必要とするバージョンよりだいぶ高いのでアンインストールしてから他の依存モジュールをインストールします.

In [ ]:
# pythonのモジュールのインストール (ランタイムが変わるたびに必要)
!pip uninstall --yes tensorflow 
!python -m pip install -r "{DFL_DIR}/requirements-colab.txt"

In [ ]:
# tensorflowの確認
!pip freeze | grep tensorflow

諸々の依存関係が解決出来たところでtensorflowでGPUがきちんと使えるか確認します.

In [ ]:
# tensorflow-gpu の動作確認
from tensorflow.python.client import device_lib
device_lib.list_local_devices()


これで環境構築は終了です.


# DeepFaceLabを動かしてみる

## 動画のアップロード

ここから実際にDFLを動かしていきます.

まずは5〜10秒程度の人の顔が映った動画を2つ準備します.

一つは顔が変化する対象の動画で, もう一つは実際に合成される顔の元になる動画です.

* 前者を `data_dst.mp4` or `data_dst.mov`
* 後者を `data_src.mp4` or `data_src.mov`

と名前を付けて Google Drive のマイドライブ内に作られた `DFL/DeepFaceLab_Linux/workspace` ディレクトリの中にアップロードしてください.

## 動画から連番画像への変換

アップロードが完了したら実際に処理に移っていきます.

今回は `DeepFaceLab_Linux/scripts` 内にあるシェルスクリプトを使って実行していきます.

（このシェルスクリプト群は煩雑になりがちなpythonのスクリプト実行に必要な引数を与えて実行するだけのものなので興味がある人は中身を覗いてみると何が行われているかの理解に役立つかもしれません.）

まずは `data_src.*` と `data_dst.*` を連番の画像に変換します.

以下のスクリプトを実行すると ffmpeg を使って変換が行われます.

In [ ]:
# data_src.mov/mp4 を連番画像に展開
run_script("2_extract_image_from_data_src.sh")

In [ ]:
# data_dst.mov/mp4 を連番画像に展開
run_script("3_extract_image_from_data_dst.sh")

## 連番画像からの顔の抽出

次に, 書き出した連番画像からそれぞれの顔の部分を抽出します.

In [ ]:
# srcの連番画像から顔を抽出
run_script("4_data_src_extract_faces_S3FD.sh")

In [ ]:
# dstの連番画像から顔を抽出
run_script("5_data_dst_extract_faces_S3FD.sh")

## 抽出した顔の選別準備

上で抽出した顔はかなり精度が良いとはいえ偶に顔じゃ無いものを認識してしまったりするのでそういったデータは手動で削除します.

その際に, 適当な特徴量でソートしてあげることによって変なデータを見つけやすくするのが以下のスクリプトです.

In [ ]:
# srcの連番画像をソート
run_script("4.2_data_src_sort.sh")

In [ ]:
# dstの連番画像をソート
run_script("5.2_data_dst_sort.sh")

## 抽出した顔の選別作業

Google Driveの `workspace/data_src/aligned/` と `workspace/data_dst/aligned/` ディレクトリに入っている画像を見てみましょう.

なんとなく似た順で並んでいるのが分かるかと思います. そして番号が大きい程変なデータになることが多いです.

ざっくり, 眺めつつ明らかに変なデータは削除しましょう.

この際, `data_dst` 側は誤検出以外は残していても大丈夫ですが `data_src` はなんとなくそれを使っても良い精度にならなそうな顔が欠けてしまっている画像等は容赦無く削除して良いと思います.

## 学習開始

環境構築が前準備でDFL動かし始めても前準備でしたがついに学習の開始になります.

今回は取り敢えずGoogle Colabで動いて尚且つ短期間でなんとなく顔が合成されていくのが分かるような設定を目指します.

パラメータがたくさんありますが興味があって今後自分でやってみようという人は調べると様々な情報が出てくるので参考にしてみてください.


In [ ]:
# 学習開始
run_script("6_train_SAEHD_no_preview.sh")

## 待つ

これがうまく動けばあとはひたすら待つ時間になります.

エラーが出て動かなかった人は恐らくメモリに載っていないのでバッチサイズを下げてみたり解像度を下げてみたりしてリトライしましょう.

途中経過は `workspace/model/****_SAEHD_history` 内で確認出来ます.

## 合成処理

ワクワクしながら待ったところで結果を見ていきましょう.

まずは現在のモデルを使って合成結果の連番画像を生成します.

In [ ]:
# 合成処理
run_script("7_merge_SAEHD.sh")

## 動画書き出し

上で生成された連番画像を ffmpeg を使って動画にします.

書き出しフォーマットも色々あるので気になる人は `DeepFaceLab_Linux/scripts` の中を覗いてみてください.

In [ ]:
# 動画書き出し
run_script("8_merged_to_mp4.sh")

## 学習, 学習, 学習

DeepFaceLab はきちんとしたリジューム機能が実装されているので一度確認した後にまた学習を始めることも簡単です.

設定が良い感じだと思えば前回の設定を引き継ぐことも出来ます.

もし, パラメータを変えたい場合は最初と同様1つずつ選んでいくこととなりますが, 解像度や学習するの顔のタイプ等, 途中からでは変更出来ないパラメータもありますので注意が必要です.

In [ ]:
# 再び学習開始
run_script("6_train_SAEHD_no_preview.sh")